# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="/home/maurice/Documents/udacity-project-multi-agent/data/Tennis_Linux_NoVis/Tennis.x86_64")
# env = UnityEnvironment(file_name="/home/maurice/Documents/udacity-project-multi-agent/data/Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
print('The state for the second agents looks like:', states[1])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]
The state for the second agents looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.4669857  -1.5
  0.          0.         -6.83172083  6.          0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.0


When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [6]:
from ddpg import DDPGAgent, ReplayBuffer
import numpy as np

class MADDPG:
    def __init__(self, config):
        self.config = config
        self.replay = ReplayBuffer(self.config.action_size, self.config.buffer_size, self.config.batch_size, 
                                   self.config.seed)
        self.agents = [DDPGAgent(self.config) for i in range(self.config.num_agents)]
        self.t_step = 0
        self.loss   = (0.0, 0.0)
        
        
    def reset(self):
        for agent in self.agents:
            agent.reset()
            
    
    def act(self, obs_all_agents, noise=0.0):
        """get actions from all agents in the MADDPG object"""
        actions = [agent.act(obs, noise) for agent, obs in zip(self.agents, obs_all_agents)]
        return actions
    
    
    def step(self, states, actions, rewards, next_states, dones):
        
        for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
            self.replay.add(state, action, reward, next_state, done)        
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step += 1
        if self.t_step % self.config.update_every == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.replay) > self.config.batch_size:
                closs = []
                aloss = []
                for agent in self.agents:
                    experiences = self.replay.sample()
                    critic_loss, actor_loss = agent.update(experiences, self.config.discount)
                    closs.append(critic_loss)
                    aloss.append(actor_loss)
                self.loss = (np.mean(closs), np.mean(aloss))
    
    

In [7]:
import torch
from config import Config
from collections import deque
import matplotlib.pyplot as plt

# config settings
config = Config()
config.update_every = 1
config.batch_size   = 512
config.buffer_size  = int(1e6)
config.discount     = 0.99
config.tau          = 0.2
config.seed         = 2
config.lr_actor     = 1e-4
config.lr_critic    = 1e-4
config.action_size  = action_size
config.state_size   = state_size
config.num_agents   = num_agents

config.in_actor     = state_size
config.in_critic    = state_size + num_agents * action_size
config.out_actor    = action_size

config.hidden_in_actor   = 32
config.hidden_out_actor  = 16
config.hidden_in_critic  = 32
config.hidden_out_critic = 16

maddpg = MADDPG(config)



def train(n_episodes=100):
    scores = []
    scores_window = deque(maxlen=100)
    
    for eps in range(n_episodes):
        env_info = env.reset(train_mode=True)[brain_name]
        states   = env_info.vector_observations
        maddpg.reset()
        score = np.zeros(num_agents)
        
        while True:
            actions     = maddpg.act(states)
            actions = np.clip(actions, -1, 1)               
            env_info    = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards     = env_info.rewards
            dones       = env_info.local_done
            maddpg.step(states, actions, rewards, next_states, dones)
            score      += rewards
            states      = next_states
            
            if np.any(dones): break

        scores_max    = np.max(score)
        scores_window.append(scores_max)
        scores.append(scores_max)
        
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}\tCritic Loss: {:-11.10f}\tActor Loss: {:-10.6f}\t'
              't_step {:-8d}'.
              format(eps, np.mean(scores_window), scores_max, maddpg.loss[0], 
                     maddpg.loss[1], maddpg.t_step), end="")
        
        # periodic model checkpoint        
        if eps % 50 == 0:
            torch.save(maddpg.agents[0].actor.state_dict(), 'checkpoint_actor.pth')
            torch.save(maddpg.agents[0].critic.state_dict(), 'checkpoint_critic.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}\tCritic Loss: {:-11.10f}\tActor Loss: {:-10.6f}\t'
                  't_step {:-8d}'.
                  format(eps, np.mean(scores_window), maddpg.loss[0], maddpg.loss[1], maddpg.t_step))
            
        # Stopping the training after the avg score of 30 is reached
        if np.mean(scores_window) >= 0.5:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(eps, np.mean(scores_window)))
            torch.save(maddpg.agents[0].actor.state_dict(), 'checkpoint_actor.pth')
            torch.save(maddpg.agents[0].critic.state_dict(), 'checkpoint_critic.pth')
            break

    plt.plot(np.arange(1, len(scores)+1), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

                
train()

                

Episode 0	Average Score: 0.00	Critic Loss: 0.0000000000	Actor Loss:   0.000000	t_step       14tep       14
Episode 17	Average Score: 0.00	Score: 0.00	Critic Loss: 0.0000000000	Actor Loss:   0.000000	t_step      256 next_states shape =  torch.Size([512, 24])
 next_actions shape =  torch.Size([512, 2])
len target_input2 = torch.Size([512, 26])


TypeError: forward() takes 2 positional arguments but 3 were given